In [1]:
library('SelectSim')
library('tidyverse')
library('tictoc')

── Attaching core tidyverse packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
sessionInfo()

R version 4.3.2 (2023-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /mnt/ndata/arvind/envs/R_4/lib/libopenblasp-r0.3.25.so;  LAPACK version 3.11.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Zurich
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] tictoc_1.2        lubridate_1.9.3   forcats_1.0.0     stringr_1.5.1    
 [5] dplyr_1.1.4       purrr_1.0.2       readr_2.1.5       tidyr_1.3.0      
 [9] tibble_3.2.1      ggplot2_3.4.4     tidyverse_2.0.0   SelectSim_0.0.1.3

# Metadata and Results

In [3]:
all_merged_metadata <- readRDS(file='/mnt/ndata/arvind/co_mutation_project/data/processed/all_merged_data_genie_v15.rds')

In [4]:
all_merged_result<- readRDS(file='/mnt/ndata/arvind/co_mutation_project/analysis/catalogue/results/all_merged_results_v15.rds')

In [5]:
all_significant_ids <- readRDS(file='/mnt/ndata/arvind/co_mutation_project/analysis/catalogue/results/all_merged_result_v15_significant_ids.rds')

# Analysis

In [6]:
head(all_merged_metadata,2)

,sample,class,tumor_run,oncotree,cancer_type,race,cohort,tumor_run_mapped,Tissue,final_tumor_run_mapped
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,TCGA-OR-A5J5-01,ACC,ACC,ACC,Adrenocortical Carcinoma,White,TCGA,No mapping,Adrenal Gland,ACC
2,TCGA-OR-A5JB-01,ACC,ACC,ACC,Adrenocortical Carcinoma,White,TCGA,No mapping,Adrenal Gland,ACC


In [7]:
head(all_merged_result,2)

,SFE_1,SFE_2,name,support_1,support_2,freq_1,freq_2,overlap,w_overlap,max_overlap,⋯,mean_r_nES,nFDR,cum_freq,nFDR2,type,FDR,Cohort,Panel,Tumor_run,Mapped_Tumor_run
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>
1,PIK3CA,TP53,PIK3CA - TP53,1136,3030,0.12508258,0.333627,357,317.52971,669,⋯,-7.357413,0,4166,0,ME,TRUE,TCGA,WES,Pan-can,Pan-can
2,BRAF,TP53,BRAF - TP53,611,3030,0.06727593,0.333627,82,53.39755,180,⋯,-4.784206,0,3641,0,ME,TRUE,TCGA,WES,Pan-can,Pan-can


# Pan-Can Consistent Hits Strategy
- Extract the significant hits (pair+type) at eFDR <=0.25
    - For MSK-P and DFCI-P found in all gene panel + hits found from specific panels (refer methods for more details)  
- Found in at-least 2 dataset (TCGA, MSK-P, DFCI-P)

In [9]:
# List of all extracted significant hits 
str(all_significant_ids)

List of 5
 $ TCGA  : chr [1:515] "PIK3CA - TP53_ME" "BRAF - TP53_ME" "ARID1A - TP53_ME" "BRAF - NRAS_ME" ...
 $ MSK_P : chr [1:1627] "PIK3CA - TP53_ME" "EGFR - KRAS_ME" "PTEN - TP53_ME" "ARID1A - TP53_ME" ...
 $ MSK_M : chr [1:684] "PIK3CA - TP53_ME" "EGFR - KRAS_ME" "ARID1A - TP53_ME" "GATA3 - TP53_ME" ...
 $ DFCI_P: chr [1:605] "PIK3CA - TP53_ME" "EGFR - KRAS_ME" "APC - TP53_CO" "PTEN - TP53_ME" ...
 $ DFCI_M: chr [1:425] "EGFR - KRAS_ME" "PIK3CA - TP53_ME" "ATM - TP53_ME" "BRAF - KRAS_ME" ...


In [10]:
all_merged_result_pan_can <- all_merged_result %>% filter(Tumor_run=='Pan-can')

In [11]:
dim(all_merged_result_pan_can)

[1] 445055     26

In [12]:
all_merged_result_pan_can<-all_merged_result_pan_can %>% mutate(id=case_when(SFE_1>SFE_2 ~paste(SFE_2,SFE_1,sep=" - "),
                                                  SFE_1<SFE_2 ~paste(SFE_1,SFE_2,sep=" - "))) %>% mutate(id_type=paste(id,type,sep="_"))

In [13]:
dim(all_merged_result_pan_can)

[1] 445055     28

In [14]:
head(all_merged_result_pan_can,2)

,SFE_1,SFE_2,name,support_1,support_2,freq_1,freq_2,overlap,w_overlap,max_overlap,⋯,cum_freq,nFDR2,type,FDR,Cohort,Panel,Tumor_run,Mapped_Tumor_run,id,id_type
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,PIK3CA,TP53,PIK3CA - TP53,1136,3030,0.12508258,0.333627,357,317.52971,669,⋯,4166,0,ME,TRUE,TCGA,WES,Pan-can,Pan-can,PIK3CA - TP53,PIK3CA - TP53_ME
2,BRAF,TP53,BRAF - TP53,611,3030,0.06727593,0.333627,82,53.39755,180,⋯,3641,0,ME,TRUE,TCGA,WES,Pan-can,Pan-can,BRAF - TP53,BRAF - TP53_ME


In [15]:
consistent_hit <- list('fdr_25'=list('tcga'=all_merged_result_pan_can %>% filter(nFDR2<=0.25 & Cohort=='TCGA') %>% filter(id_type %in% all_significant_ids$TCGA),
                                    'msk_p'=all_merged_result_pan_can %>% filter(nFDR2<=0.25 & Cohort=='MSK Primary') %>% filter(id_type %in% all_significant_ids$MSK_P),
                                    'msk_m'=all_merged_result_pan_can %>% filter(nFDR2<=0.25 & Cohort=='MSK Metastasis') %>% filter(id_type %in% all_significant_ids$MSK_M),
                                    'dfci_p'=all_merged_result_pan_can %>% filter(nFDR2<=0.25 & Cohort=='DFCI Primary') %>% filter(id_type %in% all_significant_ids$DFCI_P),
                                    'dfci_m'=all_merged_result_pan_can %>% filter(nFDR2<=0.25 & Cohort=='DFCI Metastasis') %>% filter(id_type %in% all_significant_ids$DFCI_M)))

In [16]:
str(consistent_hit)

List of 1
 $ fdr_25:List of 5
  ..$ tcga  :'data.frame':	222 obs. of  28 variables:
  .. ..$ SFE_1           : chr [1:222] "PIK3CA" "BRAF" "TP53" "BRAF" ...
  .. ..$ SFE_2           : chr [1:222] "TP53" "TP53" "ARID1A" "NRAS" ...
  .. ..$ name            : chr [1:222] "PIK3CA - TP53" "BRAF - TP53" "TP53 - ARID1A" "BRAF - NRAS" ...
  .. ..$ support_1       : num [1:222] 1136 611 3030 611 334 ...
  .. ..$ support_2       : num [1:222] 3030 3030 571 240 3030 3030 3030 286 571 684 ...
  .. ..$ freq_1          : num [1:222] 0.1251 0.0673 0.3336 0.0673 0.0368 ...
  .. ..$ freq_2          : num [1:222] 0.3336 0.3336 0.0629 0.0264 0.3336 ...
  .. ..$ overlap         : num [1:222] 357 82 147 13 225 158 50 162 201 16 ...
  .. ..$ w_overlap       : num [1:222] 317.53 53.4 115.19 6.35 202.01 ...
  .. ..$ max_overlap     : num [1:222] 669 180 371 191 334 385 229 193 359 125 ...
  .. ..$ freq_overlap    : num [1:222] 0.5336 0.4556 0.3962 0.0681 0.6737 ...
  .. ..$ r_overlap       : num [1:222] 485.2

In [17]:
all_merged_result_pan_can_significant <- bind_rows(consistent_hit$fdr_25$tcga,consistent_hit$fdr_25$msk_p,consistent_hit$fdr_25$dfci_p)
rownames(all_merged_result_pan_can_significant)<-c(1:nrow(all_merged_result_pan_can_significant))

In [18]:
head(all_merged_result_pan_can_significant,2)

,SFE_1,SFE_2,name,support_1,support_2,freq_1,freq_2,overlap,w_overlap,max_overlap,⋯,cum_freq,nFDR2,type,FDR,Cohort,Panel,Tumor_run,Mapped_Tumor_run,id,id_type
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,PIK3CA,TP53,PIK3CA - TP53,1136,3030,0.12508258,0.333627,357,317.52971,669,⋯,4166,0,ME,TRUE,TCGA,WES,Pan-can,Pan-can,PIK3CA - TP53,PIK3CA - TP53_ME
2,BRAF,TP53,BRAF - TP53,611,3030,0.06727593,0.333627,82,53.39755,180,⋯,3641,0,ME,TRUE,TCGA,WES,Pan-can,Pan-can,BRAF - TP53,BRAF - TP53_ME


In [19]:
pan_can_consistent_motifs<-((all_merged_result_pan_can_significant %>% select(id_type,Cohort) %>% filter(Cohort %in% c('DFCI Primary','MSK Primary','TCGA')) %>% 
distinct() %>% count(id_type) %>% filter(n>=2))$id_type)

In [20]:
pan_can_consistent_motif_v15 <- list('motif'=pan_can_consistent_motifs)

In [21]:
pan_can_consistent_motif_v15$motif_data_df<-all_merged_result_pan_can_significant %>% filter(id_type %in% pan_can_consistent_motifs) %>% filter(Cohort %in% c('DFCI Primary','MSK Primary','TCGA'))

In [22]:
df<-all_merged_result_pan_can_significant %>% filter(Cohort %in% c('DFCI Primary','MSK Primary','TCGA')) %>% 
select(id_type,Cohort,wES) %>% 
group_by(id_type) %>% filter(id_type %in% pan_can_consistent_motifs)

In [23]:
# Group by 'Group' and 'ID' columns and summarize to get maximum effect size for each group and ID combination
df_summary <- df %>%
  group_by(Cohort, id_type) %>%
  summarise(Max_Effect_Size = max(wES))

# Create a wider dataframe
effect_size_df <- pivot_wider(data = df_summary, names_from = Cohort, values_from = Max_Effect_Size) %>% arrange(id_type)

`summarise()` has grouped output by 'Cohort'. You can override using the `.groups` argument.


In [24]:
effect_size_df 

id_type,DFCI Primary,MSK Primary,TCGA
<chr>,<dbl>,<dbl>,<dbl>
AKT1 - CTNNB1_CO,7.494383,11.419997,NA
AKT1 - GATA3_CO,NA,5.713433,4.571769
AKT1 - PIK3CA_ME,-5.153629,-4.238303,-8.823168
AKT1 - PTEN_ME,-4.618466,-11.650723,-5.036551
AKT1 - TP53_ME,-4.065079,-4.484284,-6.032829
APC - ARID1A_ME,-4.571165,-3.947892,NA
APC - ARID1B_ME,-5.957359,-5.412207,NA
APC - ATM_ME,-8.163681,-10.255879,NA
APC - BCOR_ME,-5.572633,-4.221691,NA


In [25]:
effect_size_df$count <- (df %>% select(id_type,Cohort) %>% ungroup() %>% distinct() %>% count(id_type))$n

In [26]:
head(effect_size_df)

id_type,DFCI Primary,MSK Primary,TCGA,count
<chr>,<dbl>,<dbl>,<dbl>,<int>
AKT1 - CTNNB1_CO,7.494383,11.419997,NA,2
AKT1 - GATA3_CO,NA,5.713433,4.571769,2
AKT1 - PIK3CA_ME,-5.153629,-4.238303,-8.823168,3
AKT1 - PTEN_ME,-4.618466,-11.650723,-5.036551,3
AKT1 - TP53_ME,-4.065079,-4.484284,-6.032829,3
APC - ARID1A_ME,-4.571165,-3.947892,NA,2


In [27]:
mean_values <- rowMeans(effect_size_df[, c('DFCI Primary','MSK Primary','TCGA')], na.rm = TRUE)

In [28]:
effect_size_df$mean_ES <- mean_values

In [29]:
head(effect_size_df)

id_type,DFCI Primary,MSK Primary,TCGA,count,mean_ES
<chr>,<dbl>,<dbl>,<dbl>,<int>,<dbl>
AKT1 - CTNNB1_CO,7.494383,11.419997,NA,2,9.457190
AKT1 - GATA3_CO,NA,5.713433,4.571769,2,5.142601
AKT1 - PIK3CA_ME,-5.153629,-4.238303,-8.823168,3,-6.071700
AKT1 - PTEN_ME,-4.618466,-11.650723,-5.036551,3,-7.101914
AKT1 - TP53_ME,-4.065079,-4.484284,-6.032829,3,-4.860730
APC - ARID1A_ME,-4.571165,-3.947892,NA,2,-4.259529


In [30]:
pan_can_consistent_motif_v15$motif_mean_ES_df<-effect_size_df

In [31]:
str(pan_can_consistent_motif_v15)

List of 3
 $ motif           : chr [1:328] "AKT1 - CTNNB1_CO" "AKT1 - GATA3_CO" "AKT1 - PIK3CA_ME" "AKT1 - PTEN_ME" ...
 $ motif_data_df   :'data.frame':	1533 obs. of  28 variables:
  ..$ SFE_1           : chr [1:1533] "PIK3CA" "BRAF" "TP53" "BRAF" ...
  ..$ SFE_2           : chr [1:1533] "TP53" "TP53" "ARID1A" "NRAS" ...
  ..$ name            : chr [1:1533] "PIK3CA - TP53" "BRAF - TP53" "TP53 - ARID1A" "BRAF - NRAS" ...
  ..$ support_1       : num [1:1533] 1136 611 3030 611 334 ...
  ..$ support_2       : num [1:1533] 3030 3030 571 240 3030 3030 3030 286 571 684 ...
  ..$ freq_1          : num [1:1533] 0.1251 0.0673 0.3336 0.0673 0.0368 ...
  ..$ freq_2          : num [1:1533] 0.3336 0.3336 0.0629 0.0264 0.3336 ...
  ..$ overlap         : num [1:1533] 357 82 147 13 225 158 50 162 201 16 ...
  ..$ w_overlap       : num [1:1533] 317.53 53.4 115.19 6.35 202.01 ...
  ..$ max_overlap     : num [1:1533] 669 180 371 191 334 385 229 193 359 125 ...
  ..$ freq_overlap    : num [1:1533] 0.5336 

In [ ]:
saveRDS(pan_can_consistent_motif_v15,file='../results/pan_can_consistent_motif_v15_fdr_25.rds')

# Tumor Classes Consistent Hits Strategy
- Extract the significant hits (pair+type) at eFDR <=0.25
    - For MSK-P and DFCI-P found in all gene panel + hits found from specific panels (refer methods for more details)  
- Mapping of TCGA tumor runs to MSK and DFCI tumor runs
- Common Tumor groups across primary cohort
- Found in at-least 2 dataset (TCGA, MSK-P, DFCI-P)

In [32]:
all_merged_result_tumor_can <- all_merged_result %>% filter(Tumor_run!='Pan-can') %>% mutate(id=case_when(SFE_1>SFE_2 ~paste(SFE_2,SFE_1,sep=" - "),
                                                  SFE_1<SFE_2 ~paste(SFE_1,SFE_2,sep=" - "))) %>% mutate(id_type=paste(id,type,sep="_"))

In [33]:
tumor_mapped <- (all_merged_metadata %>% filter(cohort=='TCGA') %>% select(tumor_run,tumor_run_mapped) %>% distinct())$tumor_run_mapped
names(tumor_mapped)<-(all_merged_metadata %>% filter(cohort=='TCGA') %>% select(tumor_run,tumor_run_mapped) %>% distinct())$tumor_run

In [34]:
temp<-unlist((all_merged_metadata %>% filter(cohort=='TCGA') %>% select(tumor_run,tumor_run_mapped) %>% distinct())$tumor_run_mapped)

In [35]:
cleaned_tumor_groups <- tolower(gsub("[[:space:]/-]", "_", temp))

In [36]:
tumor_mapped <- cleaned_tumor_groups
names(tumor_mapped)<-(all_merged_metadata %>% filter(cohort=='TCGA') %>% select(tumor_run,tumor_run_mapped) %>% distinct())$tumor_run

In [37]:
tumor_mapped

ACC                         BLCA 
                "no_mapping"             "bladder_cancer" 
                      Breast                         CESC 
             "breast_cancer"            "cervical_cancer" 
                        CHOL                          CRC 
      "billary_tract_cancer"          "colorectal_cancer" 
                    Lymphoma                         ESCA 
   "mature_b_cell_neoplasms"     "esophagogastric_cancer" 
                          GB                         HNSC 
                    "glioma"       "head_and_neck_cancer" 
                      NCCRCC                        CCRCC 
      "renal_cell_carcinoma"       "renal_cell_carcinoma" 
                         AML                         DIFG 
                  "leukemia"                     "glioma" 
                         HCC                         Lung 
              "liver_cancer" "non_small_cell_lung_cancer" 
                      PLMESO                          SOC 
              "mesothelioma"             "ovarian_cancer" 
                        PAAD                         PCPG 
         "pancreatic_cancer"                 "no_mapping" 
                   Prostrate                         SARC 
           "prostate_cancer"        "soft_tissue_sarcoma" 
                         MEL                         STAD 
                  "melanoma"     "esophagogastric_cancer" 
                        TGCT                         THPA 
           "germ_cell_tumor"             "thyroid_cancer" 
                        THYM                         UCEC 
                "no_mapping"         "endometrial_cancer" 
                          UM 
                  "melanoma"

In [38]:
all_merged_result_tumor_can <- all_merged_result %>% filter(Tumor_run!='Pan-can') %>% mutate(id=case_when(SFE_1>SFE_2 ~paste(SFE_2,SFE_1,sep=" - "),
                                                  SFE_1<SFE_2 ~paste(SFE_1,SFE_2,sep=" - "))) %>% mutate(id_type=paste(id,type,sep="_"))

In [39]:
head(all_merged_result_tumor_can)

,SFE_1,SFE_2,name,support_1,support_2,freq_1,freq_2,overlap,w_overlap,max_overlap,⋯,cum_freq,nFDR2,type,FDR,Cohort,Panel,Tumor_run,Mapped_Tumor_run,id,id_type
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,NF1,TP53,NF1 - TP53,4,15,0.05263158,0.19736842,4,1.7289765,4,⋯,19,0.1050847,CO,TRUE,TCGA,WES,ACC,no_mapping,NF1 - TP53,NF1 - TP53_CO
2,U2AF1,BCORL1,U2AF1 - BCORL1,1,1,0.01315789,0.01315789,1,0.7692308,1,⋯,2,1.0000000,CO,FALSE,TCGA,WES,ACC,no_mapping,BCORL1 - U2AF1,BCORL1 - U2AF1_CO
3,ARID3A,ARID4A,ARID3A - ARID4A,1,1,0.01315789,0.01315789,1,0.7692308,1,⋯,2,1.0000000,CO,FALSE,TCGA,WES,ACC,no_mapping,ARID3A - ARID4A,ARID3A - ARID4A_CO
4,PIK3CA,ROS1,PIK3CA - ROS1,1,1,0.01315789,0.01315789,1,0.7692308,1,⋯,2,1.0000000,CO,FALSE,TCGA,WES,ACC,no_mapping,PIK3CA - ROS1,PIK3CA - ROS1_CO
5,RB1,DICER1,RB1 - DICER1,2,1,0.02631579,0.01315789,1,0.7692308,1,⋯,3,1.0000000,CO,FALSE,TCGA,WES,ACC,no_mapping,DICER1 - RB1,DICER1 - RB1_CO
6,ATM,MEN1,ATM - MEN1,2,6,0.02631579,0.07894737,1,1.0000000,2,⋯,8,0.7717514,CO,FALSE,TCGA,WES,ACC,no_mapping,ATM - MEN1,ATM - MEN1_CO


In [40]:
tumor_runs <- unique(all_merged_result_tumor_can$Mapped_Tumor_run)

In [41]:
length(tumor_runs)

[1] 37

In [42]:
consider_tumor_runs <- setdiff(tumor_runs,c('no_mapping'))

In [43]:
tumor_consistent_hit <- list('fdr_25'=list('tcga'=all_merged_result_tumor_can %>% filter(nFDR2<=0.25 & Mapped_Tumor_run!='no_mapping'&Cohort=='TCGA') %>% filter(id_type %in% all_significant_ids$TCGA),
                                    'msk_p'=all_merged_result_tumor_can %>% filter(nFDR2<=0.25 & Mapped_Tumor_run!='no_mapping'&Cohort=='MSK Primary') %>% filter(id_type %in% all_significant_ids$MSK_P),
                                    'msk_m'=all_merged_result_tumor_can %>% filter(nFDR2<=0.25 & Mapped_Tumor_run!='no_mapping'&Cohort=='MSK Metastasis') %>% filter(id_type %in% all_significant_ids$MSK_M),
                                    'dfci_p'=all_merged_result_tumor_can %>% filter(nFDR2<=0.25 &Mapped_Tumor_run!='no_mapping'& Cohort=='DFCI Primary') %>% filter(id_type %in% all_significant_ids$DFCI_P),
                                    'dfci_m'=all_merged_result_tumor_can %>% filter(nFDR2<=0.25 & Mapped_Tumor_run!='no_mapping'&Cohort=='DFCI Metastasis') %>% filter(id_type %in% all_significant_ids$DFCI_M)))

In [44]:
all_merged_result_tumor_can_significant <- bind_rows(tumor_consistent_hit$fdr_25$tcga,tumor_consistent_hit$fdr_25$msk_p,tumor_consistent_hit$fdr_25$dfci_p)
rownames(all_merged_result_tumor_can_significant)<-c(1:nrow(all_merged_result_tumor_can_significant))

In [45]:
head(all_merged_result_tumor_can_significant,2)

,SFE_1,SFE_2,name,support_1,support_2,freq_1,freq_2,overlap,w_overlap,max_overlap,⋯,cum_freq,nFDR2,type,FDR,Cohort,Panel,Tumor_run,Mapped_Tumor_run,id,id_type
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,RB1,TP53,RB1 - TP53,70,175,0.17587940,0.4396985,52,41.818108,70,⋯,245,0,CO,TRUE,TCGA,WES,BLCA,bladder_cancer,RB1 - TP53,RB1 - TP53_CO
2,FGFR3,TP53,FGFR3 - TP53,38,175,0.09547739,0.4396985,6,4.387238,38,⋯,213,0,ME,TRUE,TCGA,WES,BLCA,bladder_cancer,FGFR3 - TP53,FGFR3 - TP53_ME


In [46]:
consider_tumor_runs

[1] "bladder_cancer"                       
 [2] "breast_cancer"                        
 [3] "cervical_cancer"                      
 [4] "billary_tract_cancer"                 
 [5] "colorectal_cancer"                    
 [6] "mature_b_cell_neoplasms"              
 [7] "esophagogastric_cancer"               
 [8] "glioma"                               
 [9] "head_and_neck_cancer"                 
[10] "renal_cell_carcinoma"                 
[11] "leukemia"                             
[12] "liver_cancer"                         
[13] "non_small_cell_lung_cancer"           
[14] "mesothelioma"                         
[15] "ovarian_cancer"                       
[16] "pancreatic_cancer"                    
[17] "prostate_cancer"                      
[18] "soft_tissue_sarcoma"                  
[19] "melanoma"                             
[20] "germ_cell_tumor"                      
[21] "thyroid_cancer"                       
[22] "endometrial_cancer"                   
[23] "salivary_gland_cancer"                
[24] "anal_cancer"                          
[25] "appendiceal_cancer"                   
[26] "b_lymphoblastic_leukemia_lymphoma"    
[27] "skin_cancer,_non_melanoma"            
[28] "gastrointestinal_stromal_tumor"       
[29] "myeloproliferative_neoplasms"         
[30] "peripheral_nervous_system"            
[31] "bone_cancer"                          
[32] "small_cell_lung_cancer"               
[33] "uterine_sarcoma"                      
[34] "cns_cancer"                           
[35] "gastrointestinal_neuroendocrine_tumor"
[36] "sellar_tumor"

In [47]:
primary_consider_tumor_runs <- unique((all_merged_result_tumor_can_significant %>% filter(Cohort %in% c('DFCI Primary','MSK Primary','TCGA')))$Mapped_Tumor_run)

In [48]:
primary_consider_tumor_runs

[1] "bladder_cancer"                    "breast_cancer"                    
 [3] "cervical_cancer"                   "billary_tract_cancer"             
 [5] "colorectal_cancer"                 "mature_b_cell_neoplasms"          
 [7] "esophagogastric_cancer"            "glioma"                           
 [9] "head_and_neck_cancer"              "renal_cell_carcinoma"             
[11] "leukemia"                          "liver_cancer"                     
[13] "non_small_cell_lung_cancer"        "mesothelioma"                     
[15] "ovarian_cancer"                    "pancreatic_cancer"                
[17] "prostate_cancer"                   "soft_tissue_sarcoma"              
[19] "melanoma"                          "thyroid_cancer"                   
[21] "endometrial_cancer"                "salivary_gland_cancer"            
[23] "anal_cancer"                       "appendiceal_cancer"               
[25] "b_lymphoblastic_leukemia_lymphoma" "skin_cancer,_non_melanoma"        
[27] "gastrointestinal_stromal_tumor"    "myeloproliferative_neoplasms"     
[29] "bone_cancer"                       "small_cell_lung_cancer"           
[31] "uterine_sarcoma"                   "cns_cancer"

In [49]:
tcga_tumor_runs <- unique((all_merged_result_tumor_can_significant %>% filter(Cohort %in% c('TCGA')))$Mapped_Tumor_run)
msk_tumor_runs <- unique((all_merged_result_tumor_can_significant %>% filter(Cohort %in% c('MSK Primary')))$Mapped_Tumor_run)
dfci_tumor_runs <- unique((all_merged_result_tumor_can_significant %>% filter(Cohort %in% c('DFCI Primary')))$Mapped_Tumor_run)

In [50]:
all_primary_tumor_runs <- (Reduce(intersect,list(tcga_tumor_runs,msk_tumor_runs,dfci_tumor_runs)))

In [51]:
length(all_primary_tumor_runs)

[1] 17

In [52]:
all_primary_tumor_runs

[1] "bladder_cancer"             "breast_cancer"             
 [3] "billary_tract_cancer"       "colorectal_cancer"         
 [5] "esophagogastric_cancer"     "glioma"                    
 [7] "head_and_neck_cancer"       "renal_cell_carcinoma"      
 [9] "leukemia"                   "non_small_cell_lung_cancer"
[11] "ovarian_cancer"             "pancreatic_cancer"         
[13] "prostate_cancer"            "soft_tissue_sarcoma"       
[15] "melanoma"                   "thyroid_cancer"            
[17] "endometrial_cancer"

In [53]:
tumor_can_consistent_motif <- list()
all_motifs <- c()

In [54]:
for(tumor_run in all_primary_tumor_runs){
    cat('--Running--',tumor_run,"\n")
    cancer_df <- all_merged_result_tumor_can_significant %>% filter(Cohort %in% c('DFCI Primary','MSK Primary','TCGA')) %>% 
                 filter(Mapped_Tumor_run==tumor_run)
    cancer_all_motif_list <- (cancer_df%>% select(id_type,Cohort)%>% distinct() %>% count(id_type) %>% filter(n>=2))$id_type
    cancer_all <-     (cancer_df%>% select(id_type,Cohort)%>% distinct() %>% count(id_type) %>% filter(n>=3))$id_type
    cancer_atleast_two <-     (cancer_df%>% select(id_type,Cohort)%>% distinct() %>% count(id_type) %>% filter(n==2))$id_type
    if(length(cancer_all_motif_list)>=1){
        df<-cancer_df %>% select(id_type,Cohort,wES) %>% group_by(id_type) %>% filter(id_type %in% cancer_all_motif_list)
        df_summary <- df %>% group_by(Cohort, id_type) %>% summarise(Max_Effect_Size = max(wES))
        effect_size_df <- pivot_wider(data = df_summary, names_from = Cohort, values_from = Max_Effect_Size) %>% arrange(id_type)
        effect_size_df$count <- (df %>% select(id_type,Cohort) %>% ungroup() %>% distinct() %>% count(id_type))$n
        intersect_cols <- intersect(c('DFCI Primary','MSK Primary','TCGA'),colnames(effect_size_df))
        mean_values <- rowMeans(effect_size_df[, intersect_cols], na.rm = TRUE)
        effect_size_df$mean_ES <- mean_values
        tumor_can_consistent_motif[[tumor_run]]<-list('all_motif'=cancer_all_motif_list,
                                                      'motif_list'=list('all'=cancer_all,'atleast_two'=cancer_atleast_two),
                                                      'motif_data_df'=cancer_df,
                                                      'motif_mean_ES_df'=effect_size_df)
        all_motifs<-unique(c(all_motifs,cancer_all_motif_list))
    }
    cat('--Running Done--',tumor_run,"\n")
}

--Running-- bladder_cancer 


`summarise()` has grouped output by 'Cohort'. You can override using the `.groups` argument.


--Running Done-- bladder_cancer 
--Running-- breast_cancer 


`summarise()` has grouped output by 'Cohort'. You can override using the `.groups` argument.


--Running Done-- breast_cancer 
--Running-- billary_tract_cancer 


...

--Running Done-- melanoma 
--Running-- thyroid_cancer 


`summarise()` has grouped output by 'Cohort'. You can override using the `.groups` argument.


--Running Done-- thyroid_cancer 
--Running-- endometrial_cancer 


`summarise()` has grouped output by 'Cohort'. You can override using the `.groups` argument.


--Running Done-- endometrial_cancer 


In [55]:
length(unique(all_motifs))

[1] 446

In [56]:
str(tumor_can_consistent_motif,max.level = 2)

List of 16
 $ bladder_cancer            :List of 4
  ..$ all_motif       : chr [1:67] "AKT1 - TP53_ME" "ARID1A - CTNNB1_CO" "ARID1A - FAT1_ME" "ARID1A - FGFR3_ME" ...
  ..$ motif_list      :List of 2
  ..$ motif_data_df   :'data.frame':	516 obs. of  28 variables:
  ..$ motif_mean_ES_df: tibble [67 × 6] (S3: tbl_df/tbl/data.frame)
 $ breast_cancer             :List of 4
  ..$ all_motif       : chr [1:45] "AKT1 - GATA3_CO" "AKT1 - PIK3CA_ME" "AKT1 - TP53_ME" "ARID1A - TP53_ME" ...
  ..$ motif_list      :List of 2
  ..$ motif_data_df   :'data.frame':	332 obs. of  28 variables:
  ..$ motif_mean_ES_df: tibble [45 × 6] (S3: tbl_df/tbl/data.frame)
 $ billary_tract_cancer      :List of 4
  ..$ all_motif       : chr [1:6] "ARID1A - KRAS_ME" "ARID1A - TP53_ME" "BAP1 - TP53_ME" "IDH1 - KRAS_ME" ...
  ..$ motif_list      :List of 2
  ..$ motif_data_df   :'data.frame':	91 obs. of  28 variables:
  ..$ motif_mean_ES_df: tibble [6 × 5] (S3: tbl_df/tbl/data.frame)
 $ colorectal_cancer         :List of 

In [57]:
tumor_can_consistent_motif_v15<- list('tumor_types'=names(tumor_can_consistent_motif),
                                       'all_motif'=all_motifs,
                                       'tumor_run_lists'=tumor_can_consistent_motif)

In [58]:
str(tumor_can_consistent_motif_v15,max.level = 3)

List of 3
 $ tumor_types    : chr [1:16] "bladder_cancer" "breast_cancer" "billary_tract_cancer" "colorectal_cancer" ...
 $ all_motif      : chr [1:446] "AKT1 - TP53_ME" "ARID1A - CTNNB1_CO" "ARID1A - FAT1_ME" "ARID1A - FGFR3_ME" ...
 $ tumor_run_lists:List of 16
  ..$ bladder_cancer            :List of 4
  .. ..$ all_motif       : chr [1:67] "AKT1 - TP53_ME" "ARID1A - CTNNB1_CO" "ARID1A - FAT1_ME" "ARID1A - FGFR3_ME" ...
  .. ..$ motif_list      :List of 2
  .. ..$ motif_data_df   :'data.frame':	516 obs. of  28 variables:
  .. ..$ motif_mean_ES_df: tibble [67 × 6] (S3: tbl_df/tbl/data.frame)
  ..$ breast_cancer             :List of 4
  .. ..$ all_motif       : chr [1:45] "AKT1 - GATA3_CO" "AKT1 - PIK3CA_ME" "AKT1 - TP53_ME" "ARID1A - TP53_ME" ...
  .. ..$ motif_list      :List of 2
  .. ..$ motif_data_df   :'data.frame':	332 obs. of  28 variables:
  .. ..$ motif_mean_ES_df: tibble [45 × 6] (S3: tbl_df/tbl/data.frame)
  ..$ billary_tract_cancer      :List of 4
  .. ..$ all_motif       

In [ ]:
saveRDS(tumor_can_consistent_motif_v15,file='../results/tumor_can_consistent_motif_v15_fdr_25.rds')